<a href="https://colab.research.google.com/github/Dansah2/Classifying-Disaster-Tweets/blob/main/EDA_Classifying_Disaster_Tweets_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifying Disaster Tweets

Kaggle Dataset Download API Command:

kaggle competitions download -c nlp-getting-started

Use three different techniques to classify a tweet as either a 'Disaster Tweet' or 'Non-Disaster Tweet'.

##Project Outline:

1) Download the dataset

2) Explore/Analyze the data

3) Preprocess and organize the data

4) Classify using Vadar

5) Classify using Bag of Words

6) Classify using Hugging Face

## Download the Dataset

1) Install required libraries

2) Import required libraries

3) Download data from Kaggle


#### Install Required Libraries

In [ ]:
!pip install -q -U kaggle
!pip install -U numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 31.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.2 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.25.2 which is incompatible.


#### Import Required Libraries

In [ ]:
# handeling data
import numpy as np
import pandas as pd

# graphing data
pd.options.plotting.backend = "plotly"
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

# downloading data
from google.colab import drive

#### Download Data From Kaggle
https://github.com/bnsreenu/python_for_microscopists/blob/master/Tips_tricks_35_loading_kaggle_data_to_colab.ipynb

https://www.youtube.com/watch?v=yEXkEUqK52Q&t=628s

In [ ]:
# Mount google drive to store Kaggle API for future use
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# make a directory for kaggle temporary instance location in Colab
! mkdir ~/.kaggle

In [ ]:
# upload json fine to Google drive and copy the temporary location
!cp /content/drive/MyDrive/Kaggle_API/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
# change the file permissions to read/write to the owner only
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# download the kaggle data
! kaggle competitions download -c nlp-getting-started

  0% 0.00/593k [00:00<?, ?B/s]
100% 593k/593k [00:00<00:00, 104MB/s]


In [ ]:
# unzip the data
! unzip nlp-getting-started.zip

Archive:  nlp-getting-started.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
# create a function to read the data into a dataframe

def read_function(csv_file):

    return pd.read_csv(csv_file)

raw_train = read_function('/content/train.csv')
raw_test = read_function('/content/test.csv')

## Explore/Analyze the Data
1) Obtain info about the training / testing set.

2) Visulize the data.

3) Make observations about the data.


###Obtain info about the training / testing set.

Notice that all columns expect id and target (int64) contain data in the form of an object.

Training data: 7613 rows x 5 columns

Testing data: 3263 rows x 4 columns

In [ ]:
# Create method to explore the data
def exp_data(data_frame):

  print(f'Data types: \n{data_frame.dtypes}\n')
  print(f'\nData info: {data_frame.info}\n')
  print(f'\nData info:{data_frame.describe()}')

In [ ]:
exp_data(raw_train)

Data types: 
id           int64
keyword     object
location    object
text        object
target       int64
dtype: object


Data info: <bound method DataFrame.info of          id keyword location  \
0         1     NaN      NaN   
1         4     NaN      NaN   
2         5     NaN      NaN   
3         6     NaN      NaN   
4         7     NaN      NaN   
...     ...     ...      ...   
7608  10869     NaN      NaN   
7609  10870     NaN      NaN   
7610  10871     NaN      NaN   
7611  10872     NaN      NaN   
7612  10873     NaN      NaN   

                                                   text  target  
0     Our Deeds are the Reason of this #earthquake M...       1  
1                Forest fire near La Ronge Sask. Canada       1  
2     All residents asked to 'shelter in place' are ...       1  
3     13,000 people receive #wildfires evacuation or...       1  
4     Just got sent this photo from Ruby #Alaska as ...       1  
...                                                 

In [ ]:
exp_data(raw_test)

Data types: 
id           int64
keyword     object
location    object
text        object
dtype: object


Data info: <bound method DataFrame.info of          id keyword location  \
0         0     NaN      NaN   
1         2     NaN      NaN   
2         3     NaN      NaN   
3         9     NaN      NaN   
4        11     NaN      NaN   
...     ...     ...      ...   
3258  10861     NaN      NaN   
3259  10865     NaN      NaN   
3260  10868     NaN      NaN   
3261  10874     NaN      NaN   
3262  10875     NaN      NaN   

                                                   text  
0                    Just happened a terrible car crash  
1     Heard about #earthquake is different cities, s...  
2     there is a forest fire at spot pond, geese are...  
3              Apocalypse lighting. #Spokane #wildfires  
4         Typhoon Soudelor kills 28 in China and Taiwan  
...                                                 ...  
3258  EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...  
325

Note that there are a lot of null values in the 'location' column for both the training and testing sets. I will drop this column. I will also drop the 'keyword' column although it has far less null values.

In [ ]:
# Create method to count the null values in a particular column and find the percentage of that column
def count_null(data_frame, col_name):
  nulls = data_frame[col_name].isna().sum()
  rows , _ = data_frame.shape
  print(f'Nulls for {col_name}: {nulls} out of {rows} samples')

In [ ]:
count_null(raw_train, 'keyword')

Nulls for keyword: 61 out of 7613 samples


In [ ]:
count_null(raw_train, 'location')

Nulls for location: 2533 out of 7613 samples


In [ ]:
count_null(raw_test, 'keyword')

Nulls for keyword: 26 out of 3263 samples


In [ ]:
count_null(raw_test, 'location')

Nulls for location: 1105 out of 3263 samples


###Visulize the data.

Note that there is a data imbalance but it does not appear to be severe.

In [ ]:
# create a bar graph that displays the count of each class

def exp_graph_data(data_frame, target_col_name=None):

  print(f"Data shape: {data_frame.shape}\n")
  print(f'Column Names: {list(data_frame.columns)}\n')

  if target_col_name:
    class_counts = data_frame[target_col_name].value_counts()

    print(f'Label Count:\n{class_counts}')

    fig = go.Figure(go.Bar(x=class_counts.index,
                           y=class_counts.values))

    fig.update_layout(xaxis_title_text='Classes',
                      yaxis_title_text='Count',
                      title_text='Count of Each Class')
    fig.show()

In [ ]:
exp_graph_data(raw_test)

Data shape: (3263, 4)

Column Names: ['id', 'keyword', 'location', 'text']



In [ ]:
exp_graph_data(raw_train, 'target')

Data shape: (7613, 5)

Column Names: ['id', 'keyword', 'location', 'text', 'target']

Label Count:
0    4342
1    3271
Name: target, dtype: int64
